### Data Files bearbeiten und neu speichern

In [2]:
import os
def format_data(input_dir):
    ### Iteriere über Ordner und prozessiere alle Dateien mit "Wahlprogramm" im Namen
    for filename in os.listdir(input_dir):
        if "programm" in filename and "fin" not in filename and ".csv" in filename:
            print(f"Processing file: {filename}")
            
            ### Prozessierung
            with open(f"model_results_data/{filename}", "r", encoding="utf8") as file: 
                filedata = file.read()
                filedata = filedata.replace(';', ' -')
                filedata = filedata.replace('§', ';')
            ### Speichern in separate .csv file
            with open(f"plotdata/{filename[:-4]}fin.csv", 'w', encoding="utf8") as file:
                file.write(filedata)
        else:
            continue
format_data("/Users/constantin/Documents/DHBW/5-Semster/NLP/NLP/model_results_data") # delted dir with data after th use of this function

Processing file: FDP_Wahlprogramm_2021_topic_class.csv
Processing file: AFD_Wahlprogramm_2021_sentiment.csv
Processing file: SPD_Wahlprogramm_2021_topicpred_valhalla.csv
Processing file: FDP_Wahlprogramm_2021_hate.csv
Processing file: CDU-CSU_Wahlprogramm_2021_hate.csv
Processing file: DIE_LINKE_Wahlprogramm_2021_topic_class.csv
Processing file: DIE_LINKE_Wahlprogramm_2021_hate.csv
Processing file: DIE_LINKE_Wahlprogramm_2021_topicpred_valhalla.csv
Processing file: AFD_Wahlprogramm_2021_topicpred_valhalla.csv
Processing file: DIE_GRUENEN_Wahlprogramm_2021_sentiment.csv
Processing file: DIE_GRUENEN_Wahlprogramm_2021_topicpred_valhalla.csv
Processing file: CDU-CSU_Wahlprogramm_2021_topic_class.csv
Processing file: AFD_Wahlprogramm_2021_hate.csv
Processing file: DIE_GRUENEN_Wahlprogramm_2021_topic_class.csv
Processing file: FDP_Wahlprogramm_2021_topicpred_valhalla.csv
Processing file: CDU-CSU_Wahlprogramm_2021_sentiment.csv
Processing file: DIE_GRUENEN_Wahlprogramm_2021_hate.csv
Processin

### DataFrame Erstellung für Ploterstellung

In [3]:
import pandas as pd
import plotly.express as px

def gettopicpredlistvalhalla():
    topicdflist=[]
    topicpredlist=["AFD_Wahlprogramm_2021_topicpred_valhallafin.csv",
                "CDU-CSU_Wahlprogramm_2021_topicpred_valhallafin.csv",
                "DIE_GRUENEN_Wahlprogramm_2021_topicpred_valhallafin.csv",
                "DIE_LINKE_Wahlprogramm_2021_topicpred_valhallafin.csv",
                "FDP_Wahlprogramm_2021_topicpred_valhallafin.csv",
                "SPD_Wahlprogramm_2021_topicpred_valhallafin.csv"]

    for i in topicpredlist:
        topicdflist+=[pd.read_csv(f"plotdata/{i}", sep=";")]
    for i in topicdflist:
        i.drop(columns = i.columns[0], axis = 1, inplace= True)
        i.drop(columns = i.columns[0], axis = 1, inplace= True)
        i.drop(columns = i.columns[1], axis = 1, inplace= True)
        i.drop(columns = i.columns[1], axis = 1, inplace= True)
    return topicdflist

def gettopicpredlisttopic_class():
    topicdflist=[]
    topicpredlist=["AFD_Wahlprogramm_2021_topic_classfin.csv",
                "CDU-CSU_Wahlprogramm_2021_topic_classfin.csv",
                "DIE_GRUENEN_Wahlprogramm_2021_topic_classfin.csv",
                "DIE_LINKE_Wahlprogramm_2021_topic_classfin.csv",
                "FDP_Wahlprogramm_2021_topic_classfin.csv",
                "SPD_Wahlprogramm_2021_topic_classfin.csv"]
    for i in topicpredlist:
        topicdflist+=[pd.read_csv(f"plotdata/{i}", sep=";")]
    for i in range(len(topicdflist)):
        topicdflist[i].drop(columns = topicdflist[i].columns[0], axis = 1, inplace= True)
        topicdflist[i].drop(columns = topicdflist[i].columns[0], axis = 1, inplace= True)
        topicdflist[i] = topicdflist[i].rename(columns={'topic': 'labels'})
    return topicdflist

def getfindfohneSWert(topicsys):#create merged df für barabbildung ohne topicseparierung
    if topicsys=="valhalla":
        df3l=gettopicpredlistvalhalla()
    elif topicsys=="topic_class":
        df3l=gettopicpredlisttopic_class()
    partrf=["AFD","CDU-CSU","GRUENE","LINKE","FDP","SPD"]
    dfges=[]
    for i in range(len(df3l)):
        dfges+=[df3l[i]]

    #drop lines with "none" as "labels"
    for i in range(len(dfges)):
        dfges[i]=dfges[i][dfges[i]["labels"]!="none"].reset_index()

    #Erstellung finales df
    findata=[]
    for i in range(len(dfges)):
        for j in dfges[i]["labels"].unique(): #Partei, Thema, Themenanteil, Hateanteil, Sentimentpositivanteil, Sentimentnegativanteil, Sentimentneutralanteil
            #print(dfges[i][dfges[i]["Top 3"]==j])
            findata+=   [[partrf[i], 
                        j, 
                        round(dfges[i][dfges[i]["labels"]==j]["labels"].count(),6)]]
    result = pd.DataFrame(findata, columns=['Partei','Topic','Score'])
    #Generierung Prozentualer Anteile
    return result

def makecsv(df, name):
    df.to_csv(f"plotcsv/{name}", sep=";")

In [4]:
makecsv(getfindfohneSWert("valhalla"), "aggrvalhallatopicdata.csv") 
makecsv(getfindfohneSWert("topic_class"), "aggrtopic_classtopicdata.csv")

--> Plotdata dir will be deleted after this --> Plotcsv is the new dir for the plot data